In [1]:
%load_ext autoreload
%autoreload 2

import math
import os
import random
from typing import NamedTuple

In [3]:
# TODO: I think I'm over-abstracting... (not my fault, blame the Algorithm Design Manual)

Point = tuple[int, int]
Instance = list[Point]
Solution = list[int]

def distance(x: Point, y: Point):
    (x1, x2) = x
    (y1, y2) = y
    return math.sqrt(((y1 - x1) ** 2) + ((y2 - x2) ** 2))

# TODO: Is there a more efficient way to write this?
def tsp_instance_read(filename: str) -> Instance:
    with open(filename, 'r') as f:
        points = []
        n = int(f.readline())
        for _ in range(n):
            line = f.readline()
            (_, sx, sy) = line.split(' ')
            (x, y) = (float(sx), float(sy))
            points.append((x, y))
    return points 

# heuristic search helpers

def make_tsp_solution(n: int) -> Solution:
    return list(range(n))

def tsp_solution_read(filename: str) -> Solution:
    with open(filename, 'r') as f:
        perm = []
        n = int(f.readline())
        for _ in range(n):
            perm.append(int(f.readline()) - 1)
    return perm


def tsp_solution_cost(points: Instance, perm: Solution) -> float:
    assert len(points) == len(perm)

    n = len(points)

    cost = distance(points[perm[n - 1]], points[perm[0]])
    for i in range(n - 1):
        cost += distance(points[perm[i]], points[perm[i + 1]])
    return cost


# TODO: finish efficient cost change
def transition(points: Instance, perm: Solution, i: int, j: int) -> float:
    if i == j:
        return 0.0
    if i > j:
        return transition(points, perm, j, i)
    
    old_cost = tsp_solution_cost(points, perm)

    # (_, n) = s
    
    # is_neighbors = False
    # if i == (j - 1) or (i == 1 and j == n):
    #     is_neighbors = True
    
    # if i == 1 and j == n:
    #     tmp = i
    #     i = j
    #     j = tmp

    # i0 = i - 1
    # if i0 == 0:
    #     i0 = n

    # j1 = j + 1
    # if j1 > n:
    #     j1 = 1 

    # old = 0
    # if is_neighbors:
    #     old = distance(t.points[s.p[i0]], t.points[s.p[i]]) + distance(t.points[s.p[j]], t.points[s.p[j1]])
    # else:
    #     i1 = i + 1
    #     if i1 > n:
    #         i1 = 1

    #     j0 = j - 1
    #     if j0 == 0:
    #         j0 = n
    #     old = (distance(t.points[s.p[i0]], t.points[s.p[i]]) 
    #            + distance(t.points[s.p[i]], t.points[s.p[i1]])
    #            + distance(t.points[s.p[j0]], t.points[s.p[j]])
    #            + distance(t.points[s.p[j]], t.points[s.p[j1]]))

    # tmp = s.p[j]
    # s.p[j] = s.p[i]
    # s.p[i] = tmp
    tmp = perm[j]
    perm[j] = perm[i]
    perm[i] = tmp

    # new = 0
    # if is_neighbors:
    #     new = distance(t.points[s.p[i0]], t.points[s.p[i]]) + distance(t.points[s.p[j]], t.points[s.p[j1]])
    # else:
    #     i1 = i + 1
    #     if i1 > n:
    #         i1 = 1

    #     j0 = j - 1
    #     if j0 == 0:
    #         j0 = n
    #     new = (distance(t.points[s.p[i0]], t.points[s.p[i]]) 
    #            + distance(t.points[s.p[i]], t.points[s.p[i1]])
    #            + distance(t.points[s.p[j0]], t.points[s.p[j]])
    #            + distance(t.points[s.p[j]], t.points[s.p[j1]]))

    new_cost = tsp_solution_cost(points, perm)

    # assert new - old == new_cost - old_cost

    return new_cost - old_cost


# simulated annealing

REPEAT_COUNT        = 1
INITIAL_TEMPERATURE = 1.0
COOLING_STEPS       = 1000
COOLING_FRACTION    = 0.998
STEPS_PER_TEMP      = 1000
K                   = 0.01 # boltzmann constant

def anneal(points: Instance, print_every=None) -> Solution:
    perm = make_tsp_solution(len(points))

    temp = INITIAL_TEMPERATURE
    cur_cost = tsp_solution_cost(points, perm)

    step = 0
    for _ in range(COOLING_STEPS):
        temp *= COOLING_FRACTION

        beg_cost = cur_cost
        for _ in range(STEPS_PER_TEMP):
            i1 = random.randint(0, len(perm) - 1)
            i2 = random.randint(0, len(perm) - 1)

            flip = random.random()

            delta = transition(points, perm, i1, i2)
            reward = math.exp((-delta / cur_cost) / (K * temp))

            if delta < 0:
                cur_cost += delta         # accept win
            elif reward > flip:
                cur_cost += delta         # accept loss
            else:
                transition(points, perm, i1, i2) # reject

            step += 1

            if step % print_every == 0:
                # print(perm, temp, cur_cost)
                print(f'step: {step} | temp: {temp:.5f} | cost: {cur_cost:.5f}')

        if cur_cost - beg_cost < 0.0:
            temp /= COOLING_FRACTION

    return perm

def percent_difference(x, y):
    return abs(x - y) / ((x + y) / 2.0)

random.seed(546)

n = 48

points = tsp_instance_read(f'../data/tsp/tsp-{n}.txt')

approx = anneal(points, print_every=500)
approx_cost = tsp_solution_cost(points, approx)

opt = tsp_solution_read(f'../data/tsp/tsp-48-sol.txt')
opt_cost = tsp_solution_cost(points, opt)

print()
print("Optimal cost:", opt_cost)
print("Approximate cost:", approx_cost)
print("Percent difference:", round(100 * percent_difference(opt_cost, approx_cost), 5))

step: 500 | temp: 0.99800 | cost: 74541.62362
step: 1000 | temp: 0.99800 | cost: 66643.06933
step: 1500 | temp: 0.99800 | cost: 61988.91306
step: 2000 | temp: 0.99800 | cost: 60289.81966
step: 2500 | temp: 0.99800 | cost: 64393.77616
step: 3000 | temp: 0.99800 | cost: 62344.07372
step: 3500 | temp: 0.99600 | cost: 68633.58033
step: 4000 | temp: 0.99600 | cost: 68806.84817
step: 4500 | temp: 0.99401 | cost: 65324.21419
step: 5000 | temp: 0.99401 | cost: 61626.43441
step: 5500 | temp: 0.99401 | cost: 59632.18846
step: 6000 | temp: 0.99401 | cost: 66665.28328
step: 6500 | temp: 0.99202 | cost: 61059.53583
step: 7000 | temp: 0.99202 | cost: 54451.61319
step: 7500 | temp: 0.99202 | cost: 51190.58614
step: 8000 | temp: 0.99202 | cost: 44643.77309
step: 8500 | temp: 0.99202 | cost: 42287.44702
step: 9000 | temp: 0.99202 | cost: 42193.23232
step: 9500 | temp: 0.99202 | cost: 45325.15406
step: 10000 | temp: 0.99202 | cost: 47644.01913
step: 10500 | temp: 0.99004 | cost: 40440.38253
step: 11000 

Experiments:
- [ ] Supervised learning with optimal labels (bad, baseline) on TSP
- [ ] Policy gradient w/ RNN on TSP
- [ ] Active Search w/ RNN on TSP
- [ ] Combined policy gradient + active search w/ RNN on TSP
- [ ] Supervised learning with optimal labels (bad, baseline) on Knapsack
- [ ] Policy gradient w/ RNN on Knapsack
- [ ] Active Search w/ RNN on Knapsack
- [ ] Combined policy gradient + active search w/ RNN on Knapsack
- [ ] Simulated annealing? Not sure

Problems:
- [ ] **ARC-AGI** 
- [ ] **ML Compiler (super)optimization**
- [ ] **Imperative program (super)optimization**
- [ ] OEIS
- [ ] General program synthesis
    - [ ] tic-tac-toe
    - [ ] maze
    - [ ] list processing
    - [ ] symbolic regression
    - [ ] physical laws
    - [ ] sudoku
    - [ ] tsp
    - [ ] maybe nks/differentiable logic automata style synthesis?